# 1.1 DW - Estructuración y limpieza de datos

En esta etapa trabajaremos con las observaciones hechas en la etapa anterior, con el fin de generar una base de datos con los datos acordes para el análisis.
Como estamos trabajando con dos set de datos que no tienen relacion alguna, los seguiremos trabajando por separado para luego unificarlos

## Import libraries

In [9]:
# Sirve para actualizar el modulo despues de haber cambiado algo en el codigo de los modulos importados
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [42]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [11]:
# Modulos propios del proyecto que se encuentran en el directorio src
from src.utils.paths import data_dir
from src.visualization.word_cloud import words_cloud

## Set up

In [12]:
# Configuracion de opciones

# Configuramos para que muestre siempre todas las columnas disponibles
pd.options.display.max_columns = None

In [13]:
# Definimos el constantes de colores de acuerdo al negocio para usar en los plots
COLOR_VINO_TINTO = "#7C3030"
COLOR__VINO_BLANCO = "#ffffbf"

## Read Data

In [14]:
# Obtenemos la ruta de los csv's que vamos a utilizar
CHEMICAL_WINE_RAW_PATH = data_dir("raw", "wine_dataset.csv")
COMERCIAL_WINE_RAW_PATH = data_dir("raw", "wines_SPA.csv")

CHEMICAL_WINE_RAW_PATH , COMERCIAL_WINE_RAW_PATH

(WindowsPath('c:/Users/cvigo/data-projects/proyecto-final-vinos/vinos-corderhouse/data/raw/wine_dataset.csv'),
 WindowsPath('c:/Users/cvigo/data-projects/proyecto-final-vinos/vinos-corderhouse/data/raw/wines_SPA.csv'))

In [70]:
# Leemos los csv's y los guardamos en un dataframe
df_wine_chemical = pd.read_csv(CHEMICAL_WINE_RAW_PATH)
df_wine_comercial = pd.read_csv(COMERCIAL_WINE_RAW_PATH)

## Dataset Quimico

### Estructuracion

En la etapa anterior observamos que no habia muchas cosas para trabajar desde la estructuración con este dataset y que tampoco tenia valores faltantes. Lo unico que tenemos que hacer es cambiar el tipo de dato de las columna `quality` y `style` a categoricas

In [17]:
df_wine_chemical.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6497 entries, 0 to 6496
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed_acidity         6497 non-null   float64
 1   volatile_acidity      6497 non-null   float64
 2   citric_acid           6497 non-null   float64
 3   residual_sugar        6497 non-null   float64
 4   chlorides             6497 non-null   float64
 5   free_sulfur_dioxide   6497 non-null   float64
 6   total_sulfur_dioxide  6497 non-null   float64
 7   density               6497 non-null   float64
 8   pH                    6497 non-null   float64
 9   sulphates             6497 non-null   float64
 10  alcohol               6497 non-null   float64
 11  quality               6497 non-null   int64  
 12  style                 6497 non-null   object 
dtypes: float64(11), int64(1), object(1)
memory usage: 660.0+ KB


In [18]:
# Convertimos los tipos de datos de las columnas quality y style a categoricas
df_wine_chemical["quality"] = df_wine_chemical["quality"].astype("category")
df_wine_chemical["style"] = df_wine_chemical["style"].astype("category")

In [20]:
df_wine_chemical.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6497 entries, 0 to 6496
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype   
---  ------                --------------  -----   
 0   fixed_acidity         6497 non-null   float64 
 1   volatile_acidity      6497 non-null   float64 
 2   citric_acid           6497 non-null   float64 
 3   residual_sugar        6497 non-null   float64 
 4   chlorides             6497 non-null   float64 
 5   free_sulfur_dioxide   6497 non-null   float64 
 6   total_sulfur_dioxide  6497 non-null   float64 
 7   density               6497 non-null   float64 
 8   pH                    6497 non-null   float64 
 9   sulphates             6497 non-null   float64 
 10  alcohol               6497 non-null   float64 
 11  quality               6497 non-null   category
 12  style                 6497 non-null   category
dtypes: category(2), float64(11)
memory usage: 571.6 KB


Como el dataset no set no tenia valores faltantes no hay que hacer ninguna limpieza de missing values. 

In [23]:
# Guardamos el dataframe en un archivo csv
df_wine_chemical.to_csv(data_dir("intermediate", "structured_chemical.csv"), index=False)

## Dataset Comercial

### Estructuracion

De la etapa anterior identificamos para la estructuracion:
- Cambiar el tipo de la columna `year` a numerica tratando el valor `N.V.` como nulo
- Cambiar el tipo de la columna `body` a categorica
- Estandarizar los valores de la columna `rating` y cambiar el tipo a categorica

In [71]:
df_wine_comercial.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7500 entries, 0 to 7499
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   winery       7500 non-null   object 
 1   wine         7500 non-null   object 
 2   year         7498 non-null   object 
 3   rating       7500 non-null   float64
 4   num_reviews  7500 non-null   int64  
 5   country      7500 non-null   object 
 6   region       7500 non-null   object 
 7   price        7500 non-null   float64
 8   type         6955 non-null   object 
 9   body         6331 non-null   float64
 10  acidity      6331 non-null   float64
dtypes: float64(4), int64(1), object(6)
memory usage: 644.7+ KB


### year

In [72]:
# Obtenemos los valores de la columna year
df_wine_comercial.year.value_counts().sort_index()

1910      1
1922      1
1925      1
1928      1
1929      1
       ... 
2018    850
2019    331
2020    474
2021      6
N.V.    288
Name: year, Length: 71, dtype: int64

In [73]:
# Obtenemos los registros donde year es igual 'N.V.'
df_wine_comercial.loc[df_wine_comercial['year'] == 'N.V.'].head()


,winery,wine,year,rating,num_reviews,country,region,price,type,body,acidity
20,Valdespino,Toneles Moscatel,N.V.,4.8,174,Espana,Jerez-Xeres-Sherry,253.00,Sherry,4.0,3.0
133,Barbadillo,Reliquia Palo Cortado Sherry,N.V.,4.7,58,Espana,Jerez Palo Cortado,380.00,Sherry,4.0,3.0
142,Alvear,Abuelo Diego Palo Cortado,N.V.,4.7,42,Espana,Montilla-Moriles,114.28,Pedro Ximenez,5.0,1.0
143,Equipo Navazos,La Bota 78 de Oloroso,N.V.,4.7,41,Espana,Manzanilla,95.57,Sherry,4.0,3.0
267,Osborne,Solera India Oloroso Rare Sherry,N.V.,4.6,74,Espana,Jerez-Xeres-Sherry,189.99,Sherry,4.0,3.0


In [74]:
# Reemplazamos los valores 'N.V.' de year por NaN
df_wine_comercial.loc[df_wine_comercial['year'] == 'N.V.', 'year'] = np.nan

In [76]:
# Verificamos el cambio de valores de year
df_wine_comercial.loc[df_wine_comercial['year'] == 'N.V.'].head()

,winery,wine,year,rating,num_reviews,country,region,price,type,body,acidity


In [77]:
# Obtenemos los registros donde year es nan
df_wine_comercial.loc[pd.isna(df_wine_comercial['year'])].head()

,winery,wine,year,rating,num_reviews,country,region,price,type,body,acidity
20,Valdespino,Toneles Moscatel,NaN,4.8,174,Espana,Jerez-Xeres-Sherry,253.00,Sherry,4.0,3.0
46,Vega Sicilia,Unico Reserva Especial Edicion,NaN,4.7,12421,Espana,Ribera del Duero,423.50,Ribera Del Duero Red,5.0,3.0
133,Barbadillo,Reliquia Palo Cortado Sherry,NaN,4.7,58,Espana,Jerez Palo Cortado,380.00,Sherry,4.0,3.0
142,Alvear,Abuelo Diego Palo Cortado,NaN,4.7,42,Espana,Montilla-Moriles,114.28,Pedro Ximenez,5.0,1.0
143,Equipo Navazos,La Bota 78 de Oloroso,NaN,4.7,41,Espana,Manzanilla,95.57,Sherry,4.0,3.0


In [78]:
# Convertimos el tipo de dato de year a numerica
df_wine_comercial["year"] = df_wine_comercial['year'].astype('float').astype("Int32")


### body

In [82]:
df_wine_comercial.body.dtype

dtype('float64')

In [81]:
# Obtenemos los valores de la columna body
df_wine_comercial.body.value_counts().sort_index()

2.0      34
3.0     553
4.0    4120
5.0    1624
Name: body, dtype: int64

In [83]:
# convertimos el tipo de dato de body a categoria
df_wine_comercial["body"] = df_wine_comercial["body"].astype("category")

In [84]:
df_wine_comercial.body.dtype

CategoricalDtype(categories=[2.0, 3.0, 4.0, 5.0], ordered=False)